In [ ]:
def run_models(df, date_column, trade_date, env_kwargs, 
              testing_window=4,
              max_rolling_window=44):
    # 现有代码...
    
    # 检查是否有市场状态特征
    has_market_state = 'bull_market' in df.columns
    
    # 根据市场状态调整DRL模型参数
    if has_market_state:
        # 计算当前测试集中各市场状态的占比
        market_composition = {
            'bull': df[df['bull_market'] == 1].shape[0] / df.shape[0],
            'bear': df[df['bear_market'] == 1].shape[0] / df.shape[0],
            'sideways': df[df['sideways_market'] == 1].shape[0] / df.shape[0]
        }
        
        print(f"Market composition: Bull={market_composition['bull']:.2f}, Bear={market_composition['bear']:.2f}, Sideways={market_composition['sideways']:.2f}")
        
        # 根据市场状态调整超参数
        if market_composition['bear'] > 0.5:  # 熊市占比高
            # 更保守的策略参数
            A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.01, "learning_rate": 0.0001}
            PPO_PARAMS = {
                "n_steps": 2048,
                "ent_coef": 0.01,
                "learning_rate": 0.00005,
                "batch_size": 64,
            }
            DDPG_PARAMS = {"batch_size": 64, "buffer_size": 10000, "learning_rate": 0.0005}
        elif market_composition['bull'] > 0.5:  # 牛市占比高
            # 更激进的策略参数
            A2C_PARAMS = {"n_steps": 10, "ent_coef": 0.005, "learning_rate": 0.0005}
            PPO_PARAMS = {
                "n_steps": 1024,
                "ent_coef": 0.005,
                "learning_rate": 0.0002,
                "batch_size": 256,
            }
            DDPG_PARAMS = {"batch_size": 256, "buffer_size": 100000, "learning_rate": 0.002}
        else:  # 震荡市或混合
            # 平衡的策略参数
            A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
            PPO_PARAMS = {
                "n_steps": 2048,
                "ent_coef": 0.005,
                "learning_rate": 0.0001,
                "batch_size": 128,
            }
            DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}
    else:
        # 使用默认参数
        A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
        PPO_PARAMS = {
            "n_steps": 2048,
            "ent_coef": 0.005,
            "learning_rate": 0.0001,
            "batch_size": 128,
        }
        DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}
    
    # 使用调整后的参数训练模型
    a2c_model = train_a2c(agent, A2C_PARAMS)
    ppo_model = train_ppo(agent, PPO_PARAMS)
    ddpg_model = train_ddpg(agent, DDPG_PARAMS)
    # 其他模型训练...

# 修改训练函数，接受参数字典
def train_a2c(agent, params=None):
    if params is None:
        params = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
    
    model_a2c = agent.get_model(model_name="a2c", model_kwargs=params)
    trained_a2c = agent.train_model(model=model_a2c, 
                                    tb_log_name='a2c',
                                    total_timesteps=50000)
    
    return trained_a2c

# 同样修改其他训练函数...